# CSV with coordinates to Spatially Enabled Dataframe

Especially with field data collection, many times data arrives as a CSV file with sample locations defined as latitude and longitude coordinates in separate fields. Getting this data into a Spatially Enabled Dataframe and checking the results to be able to perform subsequent analysis, is fairly straightforward.

In [13]:
# minimal module imports
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import pandas as pd

In [14]:
source_csv = '../data/raw/access_putin.csv'

This time, we just consolodate it all into one step.

In [15]:
df = pd.read_csv(source_csv, index_col=0)
sdf = GeoAccessor.from_xy(df, 'coord_x', 'coord_y')
webmap = sdf.spatial.plot()
webmap.basemap = 'dark-gray-vector'
webmap

MapView(layout=Layout(height='400px', width='100%'))

ArcGIS provides a [multitude of useful services](https://developers.arcgis.com/rest/location-based-services/) [available through the Python API](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#submodules) to quickly add geograhpic context to your data as part of your data preperation. Among these is the abilty to [trace hydrology flowlines](https://developers.arcgis.com/rest/elevation/api-reference/trace-downstream.htm) and [watersheds](https://developers.arcgis.com/rest/elevation/api-reference/watershed.htm) anywhere in the world. While it does require a valid ArcGIS login (which you can get with a [free developer account](https://developers.arcgis.com/pricing/) for nonprofit use), it is free to use.

In [16]:
# get the needed modules
from arcgis.gis import GIS
from arcgis.features import hydrology

# grab the ArcGIS credentials in the config file - this is just the way I do it
import sys
sys.path.append('../src')
import config

In [17]:
# create a connection to ArcGIS Online
gis = GIS(config.arcgis.url, username=config.arcgis.username, password=config.arcgis.password)
gis

GIS @ https://geoai.maps.arcgis.com version:7.1

Just to speed up the process, we create a dataframe with a very limited set of records.

In [18]:
reach_ex = sdf[sdf.reach_id == 2208].copy()
reach_ex

,reach_id,CreationDate,Creator,EditDate,Editor,coord_x,coord_y,SHAPE
143,2208,3/12/2019 20:19:35,knu2xs,3/12/2019 20:19:35,knu2xs,-123.311162,47.403101,"{""x"": -123.31116208999998, ""y"": 47.40310066600..."


In [19]:
resp_wtrshd = hydrology.watershed(reach_ex, point_id_field='reach_id', snap_distance=500)
resp_wtrshd

ToolOutput(watershed_area=<FeatureSet> 1 features, snapped_points=<FeatureSet> 1 features)

In [20]:
resp_wtrshd.watershed_area.sdf

,AreaSqKm,DataResolution,Description,OBJECTID,PourPtID,SHAPE,Shape_Area,Shape_Length
0,158.537473,0.000832999998238,HydroSHEDS 90m,1,2208,"{""rings"": [[[-123.32166666628359, 47.392500000...",0.01891,1.041667


In [21]:
resp_wtrshd.snapped_points.sdf

,OBJECTID,PourPtID,SHAPE
0,1,2208,"{""x"": -123.3120833333333, ""y"": 47.398750000001..."


In [22]:
resp_map = resp_wtrshd.watershed_area.sdf.spatial.plot()
resp_map

MapView(layout=Layout(height='400px', width='100%'))

In [23]:
resp_map.add_layer(resp_wtrshd.snapped_points.sdf.spatial.to_featureset())